__<font size=5><center>Projeto Integrador - Sistema de Clusterização de Clientes para CRM</center></font>__

__Author__: Rui Ferreira Póvoa

__School__: Digital House Brazil

__Class__: Data Science

__Due Date__:   29/11/2019

<a name="1"></a>
# 1. Data Loading

<a name="1.1"></a>
## 1.1. Importação de Bibliotecas

In [1]:
#bibliotecas para análise de dados:
import pandas as pd                                        # tables and data manipulations
import numpy as np                                         # vectors and matrices

In [2]:
#bibliotecas para ler o arquivo e realizar análises de tipo de csv
import csv
import itertools                                           # funções úteis

In [18]:
#import of heap algoryth for sorting
import heapq

In [66]:
#biblioteca de tempo
from datetime import date, datetime, timedelta

today_date = date.today()

## 1.2. Carregando os arquivos

In [3]:
filepath = "data\Vendas.csv"

In [4]:
#Trazendo o cabeçalho:
with open(filepath, 'r', encoding = 'utf8') as csvfile:
    #head = ''.join(itertools.islice(csvfile, 1))
    #print(head)
    reader = csv.reader(csvfile)
    lst_header = next(reader)
    print(lst_header)

['id_pedido', 'id_carrinho', 'status', 'desc_status', 'id_cliente', 'id_distribuidor', 'id_sku', 'id_produto', 'produto', 'anomes_carrinho', 'data_carrinho', 'anomes_status_pedido', 'data_status_pedido', 'quantidade', 'volumepedidos', 'volumecompras', 'volumependente', 'volumecanceladorecusado', 'meio_pagamento']


In [5]:
#preparando os dados para chamar no pandas
#se vai incluir NaN não pode ser usado o "int" com i tem de ser capitalizado "I"
dtype_list = [np.int32,np.int32,np.int16,'str',np.int32,
              np.int32,'Int32','Int32','str',np.int16,
              'str','Int32','str', np.float32,np.float32,
              np.float32,np.float32,np.float32, 'str']

dtype_dict = dict(zip(lst_header, dtype_list))
print(dtype_dict)

#passando datas
parse_dates = ['data_carrinho', 'data_status_pedido']

{'id_pedido': <class 'numpy.int32'>, 'id_carrinho': <class 'numpy.int32'>, 'status': <class 'numpy.int16'>, 'desc_status': 'str', 'id_cliente': <class 'numpy.int32'>, 'id_distribuidor': <class 'numpy.int32'>, 'id_sku': 'Int32', 'id_produto': 'Int32', 'produto': 'str', 'anomes_carrinho': <class 'numpy.int16'>, 'data_carrinho': 'str', 'anomes_status_pedido': 'Int32', 'data_status_pedido': 'str', 'quantidade': <class 'numpy.float32'>, 'volumepedidos': <class 'numpy.float32'>, 'volumecompras': <class 'numpy.float32'>, 'volumependente': <class 'numpy.float32'>, 'volumecanceladorecusado': <class 'numpy.float32'>, 'meio_pagamento': 'str'}


In [6]:
#deu erro a tenar chamar os resultados com os dtypes
df = pd.read_csv(filepath,
                 sep=',',
                 header=0,
                 encoding="utf8",
                 dtype = dtype_dict,
                 parse_dates=parse_dates)

# 2. Data Pre-processing

## 2.1 Dimensionalidade do DataFrame

In [7]:
df.shape

(298702, 19)

## 2.2. Visualização primária dos dados

In [8]:
df.head(5)

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
0,107783,71935,0,Carrinho Abandonado,30696,137,4696,16544,PAPEL TOALHA INTERFOLHA 2 DOBRAS SOFT BRANCO P...,5204,2018-12-11 12:05:40.081189+00:00,NaN,NaT,4.0,59.959999,NaN,NaN,NaN,DEPÓSITO EM CONTA ANTECIPADO
1,3,3,0,Carrinho Abandonado,3,15,NaN,NaN,- BALIX,4903,2015-11-11 20:28:39.699416+00:00,NaN,NaT,1.0,22.900000,NaN,NaN,NaN,DESCONTINUADO
2,53780,33739,0,Carrinho Abandonado,3,4258,29813,1657959,CÁPSULAS DE CAFÉ ESPRESSO FORTISSIMO PARA MÁQU...,5196,2018-04-18 15:01:30.063939+00:00,NaN,NaT,1.0,14.500000,NaN,NaN,NaN,DESCONTINUADO
3,53782,33737,0,Carrinho Abandonado,4583,2900,18244,637259,ESCOVA DENTAL MACIA ESSENCIAL CLEAN UNIDADE -...,5196,2018-04-18 14:48:41.568361+00:00,NaN,NaT,1.0,1.690000,NaN,NaN,NaN,DESCONTINUADO
4,53773,33739,0,Carrinho Abandonado,3,1203,13021,61708,MACARRÃO ESPAGUETE (8) COM OVOS PACOTE 500G - ...,5196,2018-04-18 15:01:30.063939+00:00,NaN,NaT,24.0,45.119999,NaN,NaN,NaN,DESCONTINUADO


In [9]:
df.dtypes.value_counts()

float32                5
int32                  4
object                 3
Int32                  3
datetime64[ns, UTC]    2
int16                  2
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298702 entries, 0 to 298701
Data columns (total 19 columns):
id_pedido                  298702 non-null int32
id_carrinho                298702 non-null int32
status                     298702 non-null int16
desc_status                298702 non-null object
id_cliente                 298702 non-null int32
id_distribuidor            298702 non-null int32
id_sku                     231509 non-null Int32
id_produto                 270994 non-null Int32
produto                    298702 non-null object
anomes_carrinho            298702 non-null int16
data_carrinho              298702 non-null datetime64[ns, UTC]
anomes_status_pedido       52322 non-null Int32
data_status_pedido         52322 non-null datetime64[ns, UTC]
quantidade                 298536 non-null float32
volumepedidos              298401 non-null float32
volumecompras              31799 non-null float32
volumependente             827 non-null float32
volumecanceladorecusado 

Conclui que apesar de ter 298702 carrinhos criados só 52322 viram pedidos

## 2.4. Dados em Falta (NaN)

In [11]:
def missing_data(data):
    #transformando os dados num objecto Series
    total = data.isnull().sum()
    
    # calculando a percentagem por coluna
    percent = round(data.isnull().sum() / len(data) * 100, 2)
    
    #A função concat coloca uma Series/Dataframe a seguir ao outro segundo o eixo que se quiser
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    #colcando o data types das colunas numa lista
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [12]:
%%time
missing_data(df)

Wall time: 254 ms


,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
Total,0,0,0,0,0,0,67193,27708,0,0,0,246380,246380,166,301,266903,297875,279048,22714
Percent,0,0,0,0,0,0,22.49,9.28,0,0,0,82.48,82.48,0.06,0.1,89.35,99.72,93.42,7.6
Types,int32,int32,int16,object,int32,int32,Int32,Int32,object,int16,"datetime64[ns, UTC]",Int32,"datetime64[ns, UTC]",float32,float32,float32,float32,float32,object


Como existem 67193 sku não encontrados, não se consegue de forma lógica analisar o produto e sua marca, pelo que se fará um drop dessas linhas

In [13]:
df.dropna(axis=0, subset=['id_sku'], inplace = True)

In [14]:
missing_data(df)

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
Total,0,0,0,0,0,0,0,0,0,0,0,185842,185842,0,0,203542,230715,214603,19338
Percent,0,0,0,0,0,0,0,0,0,0,0,80.27,80.27,0,0,87.92,99.66,92.7,8.35
Types,int32,int32,int16,object,int32,int32,Int32,Int32,object,int16,"datetime64[ns, UTC]",Int32,"datetime64[ns, UTC]",float32,float32,float32,float32,float32,object


In [15]:
df.shape

(231509, 19)

# 3. Feature Engineering

In [67]:
aux = df[df['id_cliente'] == 3]

In [70]:
aux['T-1'] = aux['data_status_pedido'].shift(1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
aux.tail(10)

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento,T-1
298622,621,324,4,Aprovado,3,47,198,5914,DESINFETANTE COM ÁLCOOL PERFUMADO PÉROLA /PÊSS...,4995,2016-03-01 17:59:45.329917+00:00,201603,2016-03-01 17:59:45.329917+00:00,1.0,24.690001,24.690001,NaN,NaN,DESCONTINUADO,2016-03-01 17:59:45.329917+00:00
298623,621,324,4,Aprovado,3,47,451,5877,HIPOCLORITO DE SÓDIO BACTERICIDA GALÃO 5LITROS...,4995,2016-03-01 17:59:45.329917+00:00,201603,2016-03-01 17:59:45.329917+00:00,1.0,16.139999,16.139999,NaN,NaN,DESCONTINUADO,2016-03-01 17:59:45.329917+00:00
298625,621,324,4,Aprovado,3,47,37,5886,ÁGUA SANITÁRIA PREMIUM CONCENTRADO GALÃO 5LITR...,4995,2016-03-01 17:59:45.329917+00:00,201603,2016-03-01 17:59:45.329917+00:00,2.0,21.840000,21.840000,NaN,NaN,DESCONTINUADO,2016-03-01 17:59:45.329917+00:00
298626,621,324,4,Aprovado,3,47,332,5876,DETERGENTE LAVA LOUÇAS NEUTRO FRASCO 500ML - E...,4995,2016-03-01 17:59:45.329917+00:00,201603,2016-03-01 17:59:45.329917+00:00,48.0,64.800003,64.800003,NaN,NaN,DESCONTINUADO,2016-03-01 17:59:45.329917+00:00
298627,621,324,4,Aprovado,3,47,534,5926,LIMPADOR MULTIUSO TRADICIONAL GALÃO 5LITROS - ...,4995,2016-03-01 17:59:45.329917+00:00,201603,2016-03-01 17:59:45.329917+00:00,1.0,17.280001,17.280001,NaN,NaN,DESCONTINUADO,2016-03-01 17:59:45.329917+00:00
298628,566,303,4,Aprovado,3,26,39622,2988,FILÉ DE PANGASIUS CONGELADO SEM GORDURA E SEM ...,4994,2016-02-29 18:26:12.326345+00:00,201602,2016-02-29 18:26:12.326345+00:00,20.0,230.000000,230.000000,NaN,NaN,DESCONTINUADO,2016-03-01 17:59:45.329917+00:00
298651,446,256,4,Aprovado,3,26,39622,2988,FILÉ DE PANGASIUS CONGELADO SEM GORDURA E SEM ...,4994,2016-02-15 12:46:38.614071+00:00,201602,2016-02-15 12:46:38.614071+00:00,11.0,126.500000,126.500000,NaN,NaN,DESCONTINUADO,2016-02-29 18:26:12.326345+00:00
298670,294,215,4,Aprovado,3,26,36443,2989,FILÉ DE SALMÃO PREMIUM CONGELADO POR KG (FILÉS...,4994,2016-02-01 15:10:11.622224+00:00,201602,2016-02-01 15:10:11.622224+00:00,10.0,308.000000,308.000000,NaN,NaN,DESCONTINUADO,2016-02-15 12:46:38.614071+00:00
298671,294,215,4,Aprovado,3,26,39622,2988,FILÉ DE PANGASIUS CONGELADO SEM GORDURA E SEM ...,4994,2016-02-01 15:10:11.622224+00:00,201602,2016-02-01 15:10:11.622224+00:00,20.0,230.000000,230.000000,NaN,NaN,DESCONTINUADO,2016-02-01 15:10:11.622224+00:00
298677,240,195,4,Aprovado,3,26,39622,2988,FILÉ DE PANGASIUS CONGELADO SEM GORDURA E SEM ...,4993,2016-01-25 15:57:06.940667+00:00,201601,2016-01-25 15:57:06.940667+00:00,11.0,126.500000,126.500000,NaN,NaN,DESCONTINUADO,2016-02-01 15:10:11.622224+00:00


In [76]:
aux.groupby(by=['id_cliente','id_pedido']).apply(f)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


pedidos  pedidos_efetivos  pedidos_cancelados  \
id_cliente id_pedido                                                  
3          140              0                 0                   0   
           141              0                 0                   0   
           143              0                 0                   0   
           147              0                 0                   0   
           151              0                 0                   0   
...                       ...               ...                 ...   
           346268           0                 0                   0   
           346269           0                 0                   0   
           346270           0                 0                   0   
           346271           0                 0                   0   
           346272           0                 0                   0   

                      volume_compras  volume_cancelados  tk_med  \
id_cliente id_pedido                                              
3          140                   0.0                0.0     NaN   
           141                   0.0                0.0     NaN   
           143                   0.0                0.0     NaN   
           147                   0.0                0.0     NaN   
           151                   0.0                0.0     NaN   
...                              ...                ...     ...   
           346268                0.0                0.0     NaN   
           346269                0.0                0.0     NaN   
           346270                0.0                0.0     NaN   
           346271                0.0                0.0     NaN   
           346272                0.0                0.0     NaN   

                                        data_cadastro data_ult_pedido  \
id_cliente id_pedido                                                    
3          140       2015-12-14 14:48:10.199598+00:00             NaT   
           141       2015-12-14 14:48:57.322216+00:00             NaT   
           143       2015-12-14 14:49:29.136540+00:00             NaT   
           147       2015-12-14 15:59:27.860212+00:00             NaT   
           151       2015-12-14 16:04:33.697892+00:00             NaT   
...                                               ...             ...   
           346268    2019-10-28 16:13:11.527965+00:00             NaT   
           346269    2019-10-28 16:13:11.527965+00:00             NaT   
           346270    2019-10-28 16:13:11.527965+00:00             NaT   
           346271    2019-10-28 16:13:11.527965+00:00             NaT   
           346272    2019-10-28 16:13:11.527965+00:00             NaT   

                     frequencia  
id_cliente id_pedido             
3          140              NaT  
           141              NaT  
           143              NaT  
           147              NaT  
           151              NaT  
...                         ...  
           346268           NaT  
           346269           NaT  
           346270           NaT  
           346271           NaT  
           346272           NaT  

[8694 rows x 9 columns]

In [68]:
def f(x):
    d = {}
    
    d['pedidos'] = x['anomes_status_pedido'].count()
    d['pedidos_efetivos'] = x['volumecompras'].count()
    d['pedidos_cancelados'] = x['volumecanceladorecusado'].count()
    d['volume_compras'] = x['volumecompras'].sum()
    d['volume_cancelados'] = x['volumecanceladorecusado'].sum()
    d['tk_med'] = x['volumecompras'].sum() / x['volumecompras'].count()
    d['data_cadastro'] = x['data_carrinho'].min()
    d['data_ult_pedido'] = x['data_status_pedido'].max()
    d['frequencia'] = (x['data_status_pedido'] - x['data_status_pedido'].shift(1)).mean()
    
    return pd.Series(d, index=['pedidos', 'pedidos_efetivos', 
                               'pedidos_cancelados', 'volume_compras',
                               'volume_cancelados','tk_med', 'data_cadastro',
                               'data_ult_pedido','frequencia'])

df.groupby(by='id_cliente').apply(f).sort_values(by='pedidos', ascending = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


,pedidos,pedidos_efetivos,pedidos_cancelados,volume_compras,volume_cancelados,tk_med,data_cadastro,data_ult_pedido,frequencia
id_cliente,,,,,,,,,
3,8569,8296,148,518526.312500,16553.351562,62.503172,2015-12-14 14:48:10.199598+00:00,2019-10-28 15:22:54.006572+00:00,-1 days +20:09:24.945487
1072,1174,1107,67,135566.703125,7924.377930,122.463146,2016-11-16 13:26:43.433891+00:00,2019-10-29 15:28:26.643484+00:00,-1 days +01:59:28.194639
218003,954,916,38,68607.890625,2687.200684,74.899444,2019-04-02 16:16:42.394130+00:00,2019-09-11 19:40:37.309880+00:00,-1 days +20:16:14.123185
4050,757,747,10,45910.367188,1505.290039,61.459662,2018-01-17 16:04:31.236709+00:00,2019-10-15 12:04:39.781653+00:00,-1 days +04:02:01.699331
4211,747,733,14,68562.023438,3699.055908,93.536185,2018-02-23 12:15:37.023686+00:00,2019-09-28 10:13:54.830582+00:00,-1 days +05:16:44.893611
...,...,...,...,...,...,...,...,...,...
158427,0,0,0,0.000000,0.000000,NaN,2019-03-22 16:37:53.340270+00:00,NaT,NaT
158799,0,0,0,0.000000,0.000000,NaN,2019-03-22 17:06:55.413669+00:00,NaT,NaT
159264,0,0,0,0.000000,0.000000,NaN,2019-03-22 17:44:57.913975+00:00,NaT,NaT


In [60]:
df_clients = df.groupby(by='id_cliente').agg(
    
    pedidos=('anomes_status_pedido', 'count'),
    
    pedidos_efetivos = ('volumecompras', 'count'),
    
    pedidos_cancelados = ('volumecanceladorecusado', 'count'),
    
    volume_compras = ('volumecompras', sum),
    
    volume_cancelados = ('volumecanceladorecusado', sum),
    
    #tk_med = ('volumecompras', sum) / ('volumecompras', 'count'),
    
    data_cadastro = ('data_carrinho', min),
    
    data_ult_pedido = ('data_status_pedido', max)
    # Apply a lambda function
    #num_days = ('data_status_pedido', lambda x: max(x) )
    
    #(today_date if max(x) == pd.NaT else max(x)) - sorted(x)[-2]
).sort_values(by='pedidos', ascending = False)
df_clients

,pedidos,pedidos_efetivos,pedidos_cancelados,volume_compras,volume_cancelados,data_cadastro,data_ult_pedido
id_cliente,,,,,,,
3,8569,8296,148,518526.281250,16553.351562,2015-12-14 14:48:10.199598+00:00,2019-10-28 15:22:54.006572+00:00
1072,1174,1107,67,135566.703125,7924.377930,2016-11-16 13:26:43.433891+00:00,2019-10-29 15:28:26.643484+00:00
218003,954,916,38,68607.898438,2687.200439,2019-04-02 16:16:42.394130+00:00,2019-09-11 19:40:37.309880+00:00
4050,757,747,10,45910.367188,1505.290039,2018-01-17 16:04:31.236709+00:00,2019-10-15 12:04:39.781653+00:00
4211,747,733,14,68562.023438,3699.055908,2018-02-23 12:15:37.023686+00:00,2019-09-28 10:13:54.830582+00:00
...,...,...,...,...,...,...,...
158427,0,0,0,0.000000,0.000000,2019-03-22 16:37:53.340270+00:00,NaT
158799,0,0,0,0.000000,0.000000,2019-03-22 17:06:55.413669+00:00,NaT
159264,0,0,0,0.000000,0.000000,2019-03-22 17:44:57.913975+00:00,NaT


In [19]:
df_clients = df_clients.pedidos.loc[(df_clients.pedidos!=0)]

In [20]:
df_clients

id_cliente
3          8569
1072       1174
218003      954
4050        757
4211        747
           ... 
3978          1
457985        1
1418475       1
823           1
69431         1
Name: pedidos, Length: 1352, dtype: int64